In [ ]:
%matplotlib notebook

In [ ]:
!pip install -q gradio pillow numpy matplotlib

In [ ]:
# Instalamos las dependencias necesarias
!pip install -q gradio pillow numpy matplotlib

# Importamos las bibliotecas
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import requests
from io import BytesIO
from PIL import Image, ImageDraw
import gradio as gr

# URL de la imagen
url = 'https://www.builditsolar.com/Experimental/IRCamera/IR_0007.jpg'

# Descargamos la imagen
print("Descargando imagen térmica...")
response = requests.get(url)
img = Image.open(BytesIO(response.content))
img_array = np.array(img)
print(f"Imagen cargada con éxito. Dimensiones: {img_array.shape}")

# Función para procesar la imagen y extraer colores
def process_image(x, y):
    # Aseguramos que x e y sean enteros válidos
    x = int(min(max(0, x), img_array.shape[1] - 1))
    y = int(min(max(0, y), img_array.shape[0] - 1))

    # Creamos una copia de la imagen para dibujar
    img_copy = img_array.copy()

    # Dibujamos líneas horizontales y verticales para la cruz (usando numpy)
    img_copy[y, :] = [255, 0, 0]  # Línea horizontal roja
    img_copy[:, x] = [255, 0, 0]  # Línea vertical roja

    # Dibujamos un punto rojo más grande en la intersección
    for i in range(-3, 4):
        for j in range(-3, 4):
            if 0 <= y+i < img_array.shape[0] and 0 <= x+j < img_array.shape[1]:
                if i*i + j*j <= 9:  # Círculo de radio 3
                    img_copy[y+i, x+j] = [255, 0, 0]

    # Obtenemos el color RGB en esa posición
    rgb = img_array[y, x]
    r, g, b = rgb

    # Creamos una muestra de color
    color_sample = np.ones((100, 100, 3), dtype=np.uint8)
    color_sample[:, :] = rgb

    # Información de color
    hex_color = mcolors.rgb2hex(np.array([r, g, b])/255.0)
    info = f"Posición: ({x}, {y})\nRGB: ({r}, {g}, {b})\nHEX: {hex_color}"

    return img_copy, color_sample, info

# Valores iniciales (centro de la imagen)
x_initial = img_array.shape[1] // 2
y_initial = img_array.shape[0] // 2

# Creamos la interfaz de Gradio
with gr.Blocks() as demo:
    gr.Markdown("# Visualizador de Imagen Térmica con Extractor de Color RGB")

    with gr.Row():
        # Columna para la imagen principal
        with gr.Column(scale=3):
            # Mostramos la imagen procesada inicialmente
            img_processed, _, _ = process_image(x_initial, y_initial)
            img_output = gr.Image(value=img_processed, label="Imagen Térmica")

        # Columna para información de color
        with gr.Column(scale=1):
            color_preview = gr.Image(value=process_image(x_initial, y_initial)[1],
                                    label="Muestra de Color")
            color_info = gr.Textbox(value=process_image(x_initial, y_initial)[2],
                                  label="Información de Color", lines=3)

    # Controles para las coordenadas
    with gr.Row():
        x_slider = gr.Slider(0, img_array.shape[1]-1, value=x_initial, step=1,
                           label="Coordenada X")
        y_slider = gr.Slider(0, img_array.shape[0]-1, value=y_initial, step=1,
                           label="Coordenada Y")

    # Entradas numéricas para coordenadas exactas
    with gr.Row():
        x_input = gr.Number(value=x_initial, label="X", precision=0)
        y_input = gr.Number(value=y_initial, label="Y", precision=0)

    # Función para actualizar desde sliders
    def update_from_sliders(x, y):
        return process_image(x, y)

    # Función para actualizar desde entradas numéricas
    def update_from_inputs(x, y):
        x = int(min(max(0, x), img_array.shape[1] - 1))
        y = int(min(max(0, y), img_array.shape[0] - 1))
        return x, y, *process_image(x, y)

    # Conectamos los eventos
    x_slider.change(update_from_sliders, inputs=[x_slider, y_slider],
                   outputs=[img_output, color_preview, color_info])
    y_slider.change(update_from_sliders, inputs=[x_slider, y_slider],
                   outputs=[img_output, color_preview, color_info])

    x_input.change(update_from_inputs, inputs=[x_input, y_input],
                  outputs=[x_slider, y_slider, img_output, color_preview, color_info])
    y_input.change(update_from_inputs, inputs=[x_input, y_input],
                  outputs=[x_slider, y_slider, img_output, color_preview, color_info])

    # Instrucciones
    gr.Markdown("""
    ### Instrucciones de uso:
    1. Usa los sliders para seleccionar coordenadas en la imagen
    2. O ingresa coordenadas específicas en los campos numéricos
    3. Los valores RGB y HEX se mostrarán automáticamente
    """)

# Lanzamos la aplicación
print("Iniciando la interfaz de Gradio...")
demo.launch(debug=True, share=True)

Descargando imagen térmica...
Imagen cargada con éxito. Dimensiones: (438, 511, 3)
Iniciando la interfaz de Gradio...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1a82fb5ea13d3bb1e9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://1a82fb5ea13d3bb1e9.gradio.live
